<a href="https://colab.research.google.com/github/jhermosillo/Escuela_CD_IMATE_2019/blob/master/Wiki_W2V_all.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h3><center>
    
### **Modelado de texto usando redes neuronales: algoritmo Word2Vec.**
#### Aplicación en WikiPedia para medir semejanza entre documentos.
    
</center></h3>
<h5><center>
    Dr. Jorge Hermosillo Valadez<br>
    Centro de Investigación en Ciencias<br>
    Universidad Autónoma del Estado de Morelos<br>
</center></h5>
</center>
<img src="img/logoCInC.jpg" width="100"/>
<img src="img/uaem.jpg" width="100"/>
</center>

En este curso veremos cómo:
* Los principios básicos de W2V
* Cómo construir una matriz de vectores de palabras usando W2V
* Cómo modelar documentos
* Calcular la semejanza entre dos documentos usando W2V y comparar contra PCA y LSI

# Módulos necesarios

## **Sólo para COLAB**

In [1]:
#"""
!apt-get install subversion 
!svn checkout "https://github.com/jhermosillo/Escuela_CD_IMATE_2019/trunk/datos/"
!svn checkout "https://github.com/jhermosillo/Escuela_CD_IMATE_2019/trunk/modelos/"
#"""

"apt-get" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.
"svn" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.
"svn" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.


In [2]:
#"""
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
#from google.colab import auth
from oauth2client.client import GoogleCredentials

#auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
file_drive = GoogleDrive(gauth)
#"""

ApplicationDefaultCredentialsError: The Application Default Credentials are not available. They are available if running in Google Compute Engine. Otherwise, the environment variable GOOGLE_APPLICATION_CREDENTIALS must be defined pointing to a file defining the credentials. See https://developers.google.com/accounts/docs/application-default-credentials for more information.

La siguiente instrucción requiere el vínculo al archivo desde DRIVE

In [ ]:
wi = file_drive.CreateFile({'id':'1sV6vK0CLUXpH1VInmtBUBnVACMm5fPFB'})

NameError: name 'file_drive' is not defined

In [ ]:
wi.GetContentFile('wiki.py')

NameError: name 'wi' is not defined

In [ ]:
import wiki as wi

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\LABSEMCO\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [ ]:
import numpy as np
import glob

import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\LABSEMCO\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


In [ ]:
archivos = glob.glob('./datos/textosWiki_1')
print(archivos)

['./datos/textosWiki_1']


Leemos los archivos descargados y sus nombres.

In [ ]:
file,nombres = wi.carga_datos(archivos)

leyendo...
./datos/textosWiki_1
tamaño del contenido de archivos cargados:             12 MB


# **Extracción de documentos**

In [ ]:
docs = wi.lee_documentos(file,nombres)
print('Se leyeron {} archivos'.format(len(docs)))
print(docs[0][0][0],docs[0][0][1][:100])

error: missing ), unterminated subpattern at position 0

# **Data Frame de documentos**

In [ ]:
# df,documentos=wi.get_dataFrame_WiDocs(docs)
# print(df.shape)
# df.head()

df_0 = pd.DataFrame(docs[0],columns = ['doc_id','Texto','clase'])
#df_1 = pd.DataFrame(docs[1],columns = ['doc_id','Texto','clase'])
print(len(df_0.index),'documentos clase 0')
#print(len(df_1.index),'documentos clase 1')
df = df_0
#df = pd.concat([df_0, df_1], ignore_index=True, sort=False)
print(df.shape)
df.head()

NameError: name 'docs' is not defined

# **Modelo Word2Vec**

**Introducción** <br>

El modelo Word2Vec (Mikolov et al., 2013) es un algoritmo de representación latente (o embebida) de palabras, que se calcula utilizando una red neuronal.

Su origen epistémico está en los modelos estadísticos del lenguaje.

$P(w_1,w_2,\cdots,w_n)=P(w_1)P(w_2|w_1)P(w_3|w_1^2)\cdots P(w_n|w_1^{n-1})$.

Estos modelos, buscan calcular la probabilidad de _n-gramas_: $P(w_1)$ unigramas, $P(w_2|w_1)$ bigramas, $P(w_3|w_1^2)$ trigramas, etc.

Los unigramas, son modelos tipo Bolsa-de-Palabras, ya que todas las palabras se consideran _independientes_; los bigramas son modelos donde se busca la probabilidad de una palabra, dado un _contexto_ de una palabra; en los trigramas el contexto es de dos palabras, y así sucesivamente.

El uso pionero de redes neuronales para calcular estas probabilidades se debe a Bengio y colegas (Bengio et al., 2003). La hipótesis es que _**los términos que co-ocurren en contextos similares tendrán representaciones similares**_, ya que la red neuronal busca maximizar el valor de probabilidad de co-ocurrencia y ajusta los pesos (representación embebida) de la red para este fin. 

Sin embargo, la red de Bengio era profunda y muy ineficiente. La aportación de Mikolov y colegas fue optimizar la arquitectura, haciéndola superficial y utilizando trucos de aceleración del cómputo.

**Modelos en word2vec** <br>

Word2vec implementa dos tipos de modelos: CBOW (Continuous Bag-of-Words: predicción de una palabra dado un contexto de n palabras) y SKIP-gram (predicción de un contexto de n palabras, dada una palabra).

</center>
<img src="img/CBOW.png" width="300"/>
</center><em><center>Modelo CBOW de un bigrama</em></center>

</center>
<img src="img/SKIP-gram.png" width="300"/>
</center><em><center>Modelo SKIP-gram para un contexto de 3 palabras</em></center>

**Paso hacia adelante (forward propagation) en word2vec** <br>

Las palabras del vocabulario se modelan como un vector _one-hot_, donde sola hay un $1$ en la unidad correspondiente a la palabra de entrada.

Si el vocabulario es de tamaño $V$, y si la capa oculta ($\mathbf{h}$) tiene $N$ neuronas, entonces la matriz de pesos que une la entrada a $\mathbf{h}$, $\mathbf{W}$, es de tamaño $V\times N$. 

Cada fila de $\mathbf{W}$ es la representación vectorial $\mathbf{v}_w$ de dimensión $N$ de la palabra $w$. Formalmente, la fila $i$ de $\mathbf{W}$ es $\mathbf{v}^{^\textrm{T}}_w$.

Dado un contexto (de una palabra para el modelo CBOW del bigrama), suponiendo $x_k=1$ y $x_k'=0$ para $k'\neq k$ tenemos:
<center>
$\mathbf{h}=\mathbf{W}^{^\textrm{T}}\mathbf{x}=\mathbf{W}^{^\textrm{T}}_{(k,\cdot)}:=\mathbf{v}^{^\textrm{T}}_{w_I}$
</center>

Hacia la salida, hay otra matriz $\mathbf{W}'$ de tamaño $N\times V$, por lo que una unidad de salida $j$ (palabra del vocabulario) tendrá un puntaje (score):
<center>
$u_j=\mathbf{v}_{w_j}'^{^\textrm{T}}\mathbf{h}$
</center>
donde $\mathbf{v}_{w_j}'$ es la columna $j$ de la matriz $\mathbf{W}'$.

Para obtener la distribución a posteriori de las palabras del vocabulario, que es una distribución multinomial, podemos usar un modelo de clasificación multiclase log-lineal llamado _softmax_ 
<center>
$p(w_j|w_I)=y_j=\frac{\exp(u_j)}{\sum_{j'=1}^{V}\exp(u_{j'})}$
</center>

El objetivo es entonces optimizar la expresión anterior mediante el algoritmo de descenso de gradiente, utilizando una función de costo (loss function) de entropía cruzada. 

**Entropía cruzada y cálculo de pesos en word2vec** <br>

**Entropía:**<br>

Recordemos que podemos medir la cantidad de información de un evento estocástico dada su probabilidad:
<center>
$I(E)=-\log[Pr(E)]=−\log(p)$
</center>

La entropía es el valor esperado o promedio de la información de un conjunto de eventos estocásticos.

El valor esperado de una variable aleatoria se escribe:
<center>
$E[X]=\sum_i^nx_ip_i$
</center>
Por lo que la entropía es:
<center>
$E[I(X)]=E[-\log[Pr(I(X))]]=-\sum_i^np(x_i)\log p(x_i)$
</center>


**Entropía cruzada:**<br>

Una forma de interpretar la entropía cruzada es verla como (menos) una función de verosimilitud log (log-likelyhood) para datos $y_i'$, bajo un modelo $y_i$.

Es decir, supongamos que tenemos algún modelo fijo (también conocido como "hipótesis"), que predice para $n$ clases $\{1,2, \cdots,n\}$ sus probabilidades de ocurrencia hipotéticas $y_1, y_2, \cdots, y_n$. Supongamos que ahora observamos (en realidad) $k_1$ instancias de la clase 1, $k_2$ instancias de la clase 2, $k_n$ instancias de la clase $n$, etc. 

Según el modelo, la probabilidad de que esto ocurra es (distribución multinomial):
<center>
$P[datos|modelo]:= y_1^{k_1}\,y_2^{k_2}\,\cdots,y_n^{k_n}$.
</center>

Tomando el logaritmo y cambiando el signo:
<center>
$-\log\,P[datos|modelo]= -k_1\log\,y_1-k_2\log\,y_2\,\cdots,-k_n\log\,y_n=-\sum_i k_i\log\,y_i$.
</center>

Si ahora dividimos por el número de observaciones $N=k_1+k_2+\cdots+k_n$, y escribimos las probabilidades empíricas $y_i'=k_i/N$, tenemos la _entropía cruzada_:
<center>
$-\frac{1}{N}\log\,P[datos|modelo]= -\frac{1}{N}\sum_i k_i\log\,y_i = \sum_i y_i'\log\,y_i $.
</center>

En el caso de las redes neuronales, $y_i'$ corresponde con el valor _verdadero_ de la instancia ($\{0,1\}$) y $y_i$ es el valor que predice el modelo. 


</center>
<img src="img/softmax.png" width="450"/>
</center><em><center>Resumen del modelo CBOW - Forward pass</em></center>

**Funciones de costo en word2vec**(Rong, 2014)<br>

En word2vec queremos maximizar:
<center>
$p(w_O|w_I)=y_{j*}=\frac{\exp(u_{j*})}{\sum_{j'=1}^{V}\exp(u_{j'})}$
</center>
donde $j*$ es el índice de la palabra que debe estar a la salida. Este es el índice que se compara contra la salida de la red cuando se entrena.

**CBOW:**<br>
<center>
$E=-\log p(w_O|w_I)=-u_{j*}+\log\sum_{j'=1}^{V}\exp(u_{j'})$
</center>
<center>
$E=-\mathbf{v}_{w_O}'^{^\textrm{T}}\cdot\mathbf{h}+\log\sum_{j'=1}^{V}\exp(\mathbf{v}_{w_j}'^{^\textrm{T}}\cdot\mathbf{h})$
</center>


En el caso en que haya $C$ palabras de entrada (e.g. trigramas o más) la expresión de arriba es la misma, solo cambia $\mathbf{h}$ que en este caso es:
<center>
$\mathbf{h}=\frac{1}{C}\mathbf{W}^{^\textrm{T}}(\mathbf{x}_1+\mathbf{x}_2+\cdots+\mathbf{x}_C)$
</center>

**SKIP-gram**<br>
Para este caso, en lugar de tener una sola distribución multinomial, tenemos $C$ distribuciones, donde $C$ es el número de palabras del contexto.
<center>
$E=-\log p(w_{O,1},w_{O,2},\cdots,w_{O,C}|w_I)=-\log\,\prod_{c=1}^C\frac{\exp(u_{c,j*})}{\sum_{j'=1}^{V}\exp(u_{j'})}=
-\sum_{c=1}^{C}u_{j*_c}+C\cdot\log\sum_{j'=1}^{V}\exp(u_{j'})$
</center>


De esta forma, se puede aplicar el algoritmo de Back-Propagation, donde se calcula el gradiente de las funciones de costo con respecto a las entradas, según el caso y según la capa correspondiente.

## **Gensim**

Vamos a utilizar las librerías y módulos de [gensim](https://radimrehurek.com/gensim/) para [word2vec](https://radimrehurek.com/gensim/models/word2vec.html#gensim.models.word2vec.Word2Vec), que tienen una amplia gama de soluciones en Python para procesamiento de la [Wikipedia](https://radimrehurek.com/gensim/scripts/segment_wiki.html), y el Procesamiento de Lenguaje Natural en general.

In [ ]:
types=df['Texto'].str.split(' ',expand=True).stack().unique()
Textos=df.Texto.values

#Creamos las oraciones, este será la entrada del modelo W2V
frases = [s.split() for s in Textos]

documentos= []

#Concatenamos todas las oraciones en una sola lista
for f in frases:
    documentos.append(f)

print('Hay {} documentos y {} palabras únicas'.\
      format(len(documentos),len(types)))

Hay 4753 documentos y 103132 palabras únicas


In [ ]:
from gensim.models import Word2Vec

vec_dim= 300
W2V = Word2Vec(documentos, min_count=1, workers=4, window=5)

In [ ]:
print(W2V.wv.get_vector("comenzó"))

[-0.95365494  1.1842033   0.95986784 -0.20210184  0.06581045 -1.5050063
  0.3009835   3.0103698  -0.39941457 -0.67336667  0.25975367 -1.7849637
 -0.6012992   1.0028936   0.78561294  0.07730709  0.4696674  -0.48145908
 -0.00316865 -1.9760407  -0.08393261  0.11784887  1.2656853  -0.5249868
  0.3182674   0.45680985 -0.4472887  -0.32462436 -0.92281055  0.28334874
  1.1180803   0.06162243 -0.08688541 -0.9917644  -0.26961812  0.40924034
  0.4008505  -1.2847699  -0.56720024 -1.4302909   0.28427872 -0.7403414
 -0.71390074  0.02349045  0.98195964 -0.06941931 -1.1503823   0.03580956
 -0.20438397  0.8060666   0.34001172 -1.420007   -1.3811796  -0.25946775
 -0.6504957   0.26506352  1.0957309  -0.03950319 -0.9075906   0.29268694
 -0.4622718   0.53714085 -0.68410915  0.21584918 -0.528052    1.075222
  1.117179    0.7701692  -0.75786674  1.1643965  -0.40643275  0.71804243
  1.0749515  -0.47401437  0.57457703  0.57578593  0.10439728 -0.39164957
 -1.1636435  -0.05190044 -0.24959731 -0.837277   -0.61710

Podemos guardar el modelo

In [ ]:
W2V.save('datos/word2vec.model')

o leer el modelo

In [ ]:
W2V= Word2Vec.load('datos/word2vec.model')

In [ ]:
print(W2V.wv.get_vector("comenzó")[:10])

[-0.95365494  1.1842033   0.95986784 -0.20210184  0.06581045 -1.5050063
  0.3009835   3.0103698  -0.39941457 -0.67336667]


### Modelación de documentos

In [ ]:
from collections import Counter, OrderedDict

def modela_documentos_rep(df):
    id_=df.doc_id.values
    datos=df.drop(columns=['doc_id'])
    datos=datos.values
    dx=[]
    for i,doc_id in enumerate(id_):
        dx.append((doc_id,datos[i]))
    do=pd.DataFrame(dx,columns=['doc_id','Vectores'])
    return do

def modela_documentos(df,w2v):
    docs=df.doc_id.values
    textos=df.Texto.str.split(' ').values.tolist()
    d=[]
    Dx=[]
    for i,texto in enumerate(textos):
        for w in texto:
            e=w2v.wv.get_vector(w)
            d.append(e)
        d=np.array(d)
        dx=np.sum(d,axis=0)/len(d)
        Dx.append([docs[i],dx])
        d=[]
    do=pd.DataFrame(Dx,columns=['doc_id','W2V'])
    return do

def k_vecinos_mas_cercanos(docus,df,k=1):
    l=docus.doc_id.values
    vec=OrderedDict()
    for id_ in l:
        d=dist_vecinos(id_,df)
        for i in range(k):
            if i==0:
                vec[id_]=[[d[i][1],d[i][2]]]
            else:
                vec[id_].append([d[i][1],d[i][2]])
    return vec

def vecinos_mas_cercanos(df,distancias):
    l=df.doc_id.values
    vec=OrderedDict()
    for id_ in l:
        for i,d in enumerate(distancias):
            if id_ == d[0]:
                vecino=d[1]
                if id_ not in vec.keys():
                    vec[id_]=[(vecino,d[2])]
                else:
                    vec[id_].append((vecino,d[2]))
            elif id_== d[1]:
                vecino=d[0]
                if id_ not in vec.keys():
                    vec[id_]=[(vecino,d[2])]
                else:
                    vec[id_].append((vecino,d[2]))
    return vec

def dist_vecinos(id_docu,df):
    dist=[]
    candidato = df[df['doc_id']==id_docu]
    candidato = candidato.iloc[:,1].values[0]
    fila=df.index[df['doc_id'] == id_docu].tolist()
    pts=df.drop(df.index[fila])
    id_=pts.doc_id.values
    pts=pts.iloc[:,1].values

    for i in range(len(pts)):
        d = np.sqrt(np.sum(np.square(candidato-pts[i])))
        dist.append((id_docu,id_[i],d))
    dist=sorted(dist,key=lambda x: x[2])
    return dist

In [ ]:
edf=modela_documentos(df,W2V)

## **Ejercicio 1**

Queremos saber que tan bien podemos modelar documentos utilizando Word2Vec.

Compara estos resultados con los obtenidos por les métodos PCA y LSA.

### Documentos de análisis

In [ ]:
# docus = df[(df['doc_id']=='1023628') |\
#            (df['doc_id']=='1024447') |\
#            (df['doc_id']=='1035967') |\
#            (df['doc_id']=='1891029') |\
#            (df['doc_id']=='1894599') ]  
docus = df.sample(n=5)
docus.index=range(len(docus.index))

docus.head()

,doc_id,Texto,clase
0,1885635,ornithopus perpusillus planta familia fabáceas...,0
1,1886681,frivolité variedad curiosa encaje pasamanería ...,0
2,1890585,dekker apellido puede referirse siguientes per...,0
3,1877195,vida suficiente segundo álbum banda mexicali i...,0
4,1874126,guyencourtsurnoye población comuna francesa re...,0


In [ ]:
k=1
vecinos=k_vecinos_mas_cercanos(docus,edf,k)

In [ ]:
print(vecinos)

OrderedDict([('1885635', [['1885583', 0.23433523]]), ('1886681', [['1889858', 0.11623342]]), ('1890585', [['1894945', 0.2674673]]), ('1877195', [['1885993', 0.20900859]]), ('1874126', [['1874168', 0.006189191]])])


In [ ]:
lista = []
for item in vecinos:
    vecino = vecinos[item][0][0]
    lista.append((item,vecino))
for docs in lista:
    print(df.loc[df['doc_id']==docs[0]].Texto.values)
    print('  ',df.loc[df['doc_id']==docs[1]].Texto.values)
    print()
    

['ornithopus perpusillus planta familia fabáceas descripción pelosa anual tallos extendidos hojas pares folíolos pequeños elípticos oblongos flores blancas rosas cabezuelas flores brácteas bajo cabezuela largas flores lóbulos dientes cáliz mitad largo tubo vaina constreñida segmentos pico recto ganchudo florece primavera otoño hábitat habita lugares arenisca arena distribución bélgica gran bretaña dinamarca francia alemania irlanda holanda suecia suiza españa italia portugal polonia rumanía enlaces externos']
   ['cuernecillo grande lotus uliginosus especie botánica erecta ascendente pelosa perenne tallos huecos hojas folíolos obovados verdiazules debajo flores amarillas menudo teñidas rojo cabezuelas largo cabillo normalmente flores pétalos dientes cáliz extendidos brote aproximadamente largo tuco vaina florece mayo agosto habita pantanos prados húmedos gran parte europa introducida noruega finlandia hungría centro españa habita comunidades quercofraxinetum enlaces externos']

['frivo

## **Ejercicio 2**

Modelo SKIP-gram... Ojo! Es tardado...

In [ ]:
from gensim.models import Word2Vec

vec_dim= 300
w2v_sg = Word2Vec(documentos, min_count=1, size=vec_dim, workers=4, window=5, iter=30,sg=1)
print(W2V["comenzó"][:10])

In [ ]:
w2v_sg=Word2Vec.load('/content/gdrive/My Drive/MisCursos/word2vec_sg.model')

In [ ]:
edf_sg=wi.modela_documentos(df,w2v_sg)

In [ ]:
k=1
vecinos_sg=wi.k_vecinos_mas_cercanos(docus,edf_sg,k)

In [ ]:
print(vecinos_sg['1891029'])

In [ ]:
test1=df[df['doc_id']=='1891029'].values
test2=df[df['doc_id']=='1896707'].values
print(test1)
print(test2)

<hr>
</hr>

In [ ]:
df_lsa=pd.read_pickle('datos/data_frame_4K.pkl')
df_lsa.index = range(len(df_lsa.index))
print(df_lsa.shape)
df_lsa.head()

In [ ]:
from sklearn.feature_extraction import DictVectorizer
from collections import Counter, OrderedDict

def bow_(docs):
    v = DictVectorizer(sparse=False)
    X = v.fit_transform(docs)
    return X,v

docs = df_lsa.Conteos.tolist()
X,vocab_ = bow_(docs)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

q=300  #Elegimos usar q componentes
svd = TruncatedSVD(n_components=q, n_iter=7, random_state=42)

vectorizer = TfidfVectorizer(vocabulary=vocab_.vocabulary_)

corpus = df_lsa.Texto.tolist()
D_tfidf = vectorizer.fit_transform(corpus)

In [ ]:
dlsa=svd.fit_transform(D_tfidf)
dlsa=wi.get_dataFrame(dlsa,df_lsa)
print(dlsa.shape)
dlsa.head()

In [ ]:
svd_vr=svd.explained_variance_ratio_
wi.distribucion_vr(svd_vr)

In [ ]:
q=300  #debe ser <= 300 o debes correr de nuevo el algoritmo más arriba
lsa_rep=wi.get_representativos(dlsa,q)
print(lsa_rep.shape)
lsa_rep.head()

In [ ]:
edf_lsa=wi.modela_documentos_rep(lsa_rep)
print(edf_lsa.shape)
edf_lsa.head()

In [ ]:
k=1
vecinos_lsa=wi.k_vecinos_mas_cercanos(docus,edf_lsa,k)

In [ ]:
print(vecinos_lsa['1891029'])

In [ ]:
test1=df[df['doc_id']=='1891029'].Texto.values[0][:400]
test2=df[df['doc_id']=='1035967'].Texto.values[0][:400]
print(test1)
print(test2)

In [ ]:
print(vecinos_lsa['1023628'])

In [ ]:
test1=df[df['doc_id']=='1023628'].Texto.values[0][:400]
test2=df[df['doc_id']=='1885462'].Texto.values[0][:400]
print(test1)
print(test2)

**Referencias** <br>

Yoshua Bengio, Réjean Ducharme, Pascal Vincent, and Christian Janvin. 2003. A neural probabilistic language model. J. Mach. Learn. Res. 3 (March 2003), 1137-1155. 

Tomas Mikolov, Ilya Sutskever, Kai Chen, Greg Corrado, and Jeffrey Dean. 2013. Distributed representations of words and phrases and their compositionality. In _Proceedings of the 26th International Conference on Neural Information Processing Systems - Volume 2 (NIPS'13)_, C. J. C. Burges, L. Bottou, M. Welling, Z. Ghahramani, and K. Q. Weinberger (Eds.), Vol. 2. Curran Associates Inc., USA, 3111-3119. 

Xin Rong. 2014. Word2vec Parameter Learning Explained.arXiv 1411.2738. disponible en linea {http://arxiv.org/abs/1411.2738}